In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("Yelp/yelp_review_full", cache_dir="./data/hf/datasets")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [4]:
print(dataset["train"][0])

{'label': 4, 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}


In [2]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,4 stars,Today I had the most amazing pizza ever! I am allergic to the red sauce and so I asked for a white pizza thin crust instead. It was so delicious and fresh.\nDefinitely recommend this location and definitely will come back. Superior customs service too!
1,2 star,"Tiny RV park new Tempe, with a crotchety old owner who's actually really nice. RV parks are always defined by their price and their \""amenities\"". The price is fair, but the \""amenities\"" are nearly non-existant. Yes there's a bathroom, but it was the worst bathroom and shower that I've experienced traveling the country. Water just got everywhere, and there was no way to hang a towel or anything. Plus there was no plug in you needed to shave or dry your hair or anything. It's also not exactly where google maps says that it's located, so it's a bit difficult to find."
2,1 star,Overpriced Chinese food with little to no spices. 12$ for 4 steam dumplings that tasted bleh. Great atmosphere ok customer service but out of three dishes ordered none was good. Total disappointment. The asian food was better at planet hollywoods buffet. Big disappointment. Wont be back.
3,2 star,"I bought a coupon for $59 to play at Orange Tree and Legend Trail a few months ago. I basically just bought it to play at Legend Trail and get the Orange Tree round as a bonus. \n\nFirst off, this course is a long drive away even from the Scottsdale Airpark area. I don't mind driving this far if the experience and or value makes up for it. This certainly wasn't the case at Legend Trail.\n\nThe driving range had minimal grass that hadn't been mowed in quite some time so it was basically like practicing out of thick rough and the balls are pretty seasoned. Then when I was practicing on the chipping green, a maintenance employee pulled the flags out that I was hitting at and started mowing right in front of me. I was a little annoyed but wasn't going to let it bother me much.\n\nPlaying my round, the course was in pretty bad shape. They probably had aerated a few weeks ago would be my guess so I can't really hold that against them.\n\nWhat did annoy me though was that I was having the same issues repeatedly with maintenance staff not showing any etiquette. I had multiple greens I was hitting into from 100 yards or less and the staff would not turn off their weed whackers while I was hitting. One guy on a hole was standing 5 feet from the flag and even saw I was trying to hit and he didn't even move! There were three weed whackers going around one green while I was putting too. On another hole, I was standing over the ball to tee off and three maintenance carts drove right past me. I have never felt like such an inconvenience to staff at a golf course. I wouldn't even expect that at a course like Orange Tree.\n\nNot sure what Shane S was referring to about management understanding customer service there. I went in the pro shop afterwards to ask for a manager and the employee informed me that he was gone for the day and asked why I needed one. When I told him I was upset about the maintenance staff, he sure couldn't have cared less. I filled out an online survey letting the course know about my experience and never heard back.\n\nAll in all, not worth the drive out there. If you want a nice golf course and don't mind spending money, go to Boulders or Grayhawk. If you want a nice course but are more price sensitive, then go to We Ko Pa. It's just as far away, almost the same price for residents, and a difference of night and day for course layout, amenities and customer service there."
4,2 star,"Burgers were only warm not hot for just coming off the grill, slightly undercooked red centers. Lettuce on burgers was not good, wilted disgusting. French Fries were 70% broken stubs looks like we got the end of a the bag average again not super warm. Which I can't understand how food just coming out of kitchen is not HOT!\n\nServers were nice but when we said the meal was less than average th

In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import Trainer

In [8]:
model_name = 'google-bert/bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [9]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,"I went here for quite awhile and was always satisfied with their service. One day last summer, I walked in and saw the same lady that always did my eyebrows (I never knew her name). As we were talking, she (for whatever reason) felt the need to inform me I had put on weight and asked me if I was pregnant. I couldn't even believe her. I walked out in tears and to this day am shocked she has a job in customer service.","[101, 146, 1355, 1303, 1111, 2385, 21985, 1105, 1108, 1579, 8723, 1114, 1147, 1555, 119, 1448, 1285, 1314, 2247, 117, 146, 2045, 1107, 1105, 1486, 1103, 1269, 5141, 1115, 1579, 1225, 1139, 8039, 113, 146, 1309, 1450, 1123, 1271, 114, 119, 1249, 1195, 1127, 2520, 117, 1131, 113, 1111, 3451, 2255, 114, 1464, 1103, 1444, 1106, 12862, 1143, 146, 1125, 1508, 1113, 2841, 1105, 1455, 1143, 1191, 146, 1108, 6391, 119, 146, 1577, 112, 189, 1256, 2059, 1123, 119, 146, 2045, 1149, 1107, 3632, 1105, 1106, 1142, 1285, 1821, 6764, 1131, 1144, 170, 2261, 1107, 8132, 1555, 119, 102, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...]"


In [10]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model_dir = "./data/hf/models/bert-base-cased-finetune-yelp"
training_args = TrainingArguments(
    output_dir=model_dir, 
    per_device_train_batch_size=16, 
    num_train_epochs=5, 
    logging_strategy="epoch",
    eval_strategy="epoch"
)
print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
eval_use_gather_object=False,
fp16=False,
fp1

In [19]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/root/miniforge3/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss


/root/miniforge3/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/root/miniforge3/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/root/miniforge3/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/root/miniforge3/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=315, training_loss=0.7869944315108042, metrics={'train_runtime': 580.0302, 'train_samples_per_second': 8.62, 'train_steps_per_second': 0.543, 'total_flos': 1315590712320000.0, 'train_loss': 0.7869944315108042, 'epoch': 5.0})

In [21]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)

/root/miniforge3/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 1.1201703548431396,
 'eval_accuracy': 0.61,
 'eval_runtime': 2.7824,
 'eval_samples_per_second': 35.94,
 'eval_steps_per_second': 4.672,
 'epoch': 5.0}

In [26]:
trainer.save_model(model_dir)

In [27]:
trainer.save_state()

In [4]:
dataset = load_dataset(
    "rajpurkar/squad_v2", cache_dir="./data/hf/datasets"
)
print(dataset)
print(*[f"{k}: {v}" for k, v in dataset["train"][0].items()], sep="\n")

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})
id: 56be85543aeaaa14008c9063
title: Beyoncé
context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and

In [5]:
show_random_elements(dataset["train"], num_examples=3)

,id,title,context,question,answers
0,57304f5e2461fd1900a9cd0f,The_Blitz,"German air supremacy at night was also now under threat. British night-fighter operations out over the Channel were proving highly successful. This was not immediately apparent. The Bristol Blenheim F.1 was undergunned, with just four .303 in (7.7 mm) machine guns which struggled to down the Do 17, Ju 88, or Heinkel He 111. Moreover, the Blenheim struggled to reach the speed of the German bombers. Added to the fact an interception relied on visual sighting, a kill was most elusive even in the conditions of a moonlit sky.",How was the British night fighter operations faring?,"{'text': ['were proving highly successful.'], 'answer_start': [111]}"
1,5730f5bda5e9cc1400cdbb46,Russian_language,"According to figures published in 2006 in the journal ""Demoskop Weekly"" research deputy director of Research Center for Sociological Research of the Ministry of Education and Science (Russia) Arefyev A. L., the Russian language is gradually losing its position in the world in general, and in Russia in particular. In 2012, A. L. Arefyev published a new study ""Russian language at the turn of the 20th-21st centuries"", in which he confirmed his conclusion about the trend of further weakening of the Russian language in all regions of the world (findings published in 2013 in the journal ""Demoskop Weekly""). In the countries of the former Soviet Union the Russian language is gradually being replaced by local languages. Currently the number speakers of Russian language in the world depends on the number of Russians in the world (as the main sources distribution Russian language) and total population Russia (where Russian is an official language).","Who wrote ""Russian language at the turn of the 20th-21st centuries""?","{'text': ['A. L. Arefyev'], 'answer_start': [324]}"
2,56e1cc2fcd28a01900c67bb7,Communications_in_Somalia,"In December 2012, Hormuud Telecom launched its Tri-Band 3G service for internet and mobile clients. The first of its kind in the country, this third generation mobile telecommunications technology offers users a faster and more secure connection.",Who was the 3G service for?,"{'text': ['internet and mobile clients'], 'answer_start': [71]}"


In [6]:
model_name = 'distilbert/distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokens = tokenizer("What is your name?", "My name is Sylvain.")
print(*[f"{k}: {v}" for k, v in tokens.items()], sep="\n")

input_ids: [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [7]:
max_length = 384  # 最大文本长度
doc_stride = 128  # 截断时填充的文本长度
example = next(
    example for example in dataset["train"]
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length
)
print(*[f"{k}: {v}" for k, v in example.items()], sep="\n")

id: 56be95823aeaaa14008c910c
title: Beyoncé
context: On April 4, 2008, Beyoncé married Jay Z. She publicly revealed their marriage in a video montage at the listening party for her third studio album, I Am... Sasha Fierce, in Manhattan's Sony Club on October 22, 2008. I Am... Sasha Fierce was released on November 18, 2008 in the United States. The album formally introduces Beyoncé's alter ego Sasha Fierce, conceived during the making of her 2003 single "Crazy in Love", selling 482,000 copies in its first week, debuting atop the Billboard 200, and giving Beyoncé her third consecutive number-one album in the US. The album featured the number-one song "Single Ladies (Put a Ring on It)" and the top-five songs "If I Were a Boy" and "Halo". Achieving the accomplishment of becoming her longest-running Hot 100 single in her career, "Halo"'s success in the US helped Beyoncé attain more top-ten singles on the list than any other woman during the 2000s. It also included the successful "Sweet Drea

In [8]:
lens = len(tokenizer(example["question"], example["context"])["input_ids"])
print(lens)

437


In [9]:
lens = len(
    tokenizer(
        example["question"], 
        example["context"],
        max_length=max_length,
        truncation="only_second"
    )["input_ids"]
)
print(lens)

384


In [10]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)
print([len(x) for x in tokenized_example["input_ids"]])
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[384, 192]
[CLS] beyonce got married in 2008 to whom? [SEP] on april 4, 2008, beyonce married jay z. she publicly revealed their marriage in a video montage at the listening party for her third studio album, i am... sasha fierce, in manhattan ' s sony club on october 22, 2008. i am... sasha fierce was released on november 18, 2008 in the united states. the album formally introduces beyonce ' s alter ego sasha fierce, conceived during the making of her 2003 single " crazy in love ", selling 482, 000 copies in its first week, debuting atop the billboard 200, and giving beyonce her third consecutive number - one album in the us. the album featured the number - one song " single ladies ( put a ring on it ) " and the top - five songs " if i were a boy " and " halo ". achieving the accomplishment of becoming her longest - running hot 100 single in her career, " halo " ' s success in the us helped beyonce attain more top - ten singles on the list than any other woman during the 2000s. it also

In [11]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 7), (8, 11), (12, 19), (20, 22), (23, 27), (28, 30), (31, 35), (35, 36), (0, 0), (0, 2), (3, 8), (9, 10), (10, 11), (12, 16), (16, 17), (18, 25), (26, 33), (34, 37), (38, 39), (39, 40), (41, 44), (45, 53), (54, 62), (63, 68), (69, 77), (78, 80), (81, 82), (83, 88), (89, 93), (93, 96), (97, 99), (100, 103), (104, 113), (114, 119), (120, 123), (124, 127), (128, 133), (134, 140), (141, 146), (146, 147), (148, 149), (150, 152), (152, 153), (153, 154), (154, 155), (156, 161), (162, 168), (168, 169), (170, 172), (173, 182), (182, 183), (183, 184), (185, 189), (190, 194), (195, 197), (198, 205), (206, 208), (208, 209), (210, 214), (214, 215), (216, 217), (218, 220), (220, 221), (221, 222), (222, 223), (224, 229), (230, 236), (237, 240), (241, 249), (250, 252), (253, 261), (262, 264), (264, 265), (266, 270), (271, 273), (274, 277), (278, 284), (285, 291), (291, 292), (293, 296), (297, 302), (303, 311), (312, 322), (323, 330), (330, 331), (331, 332), (333, 338), (339, 342), (343, 3

In [12]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(example["question"])
print(
    tokenizer.convert_ids_to_tokens([first_token_id])[0], 
    example["question"][offsets[0]:offsets[1]]
)

Beyonce got married in 2008 to whom?
beyonce Beyonce


In [13]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [14]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("答案不在此特征中。")

print(tokenizer.decode(
    tokenized_example["input_ids"][0][start_position: end_position+1]
))
print(answers["text"][0])

18 19
jay z
Jay Z


In [13]:
pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)
    return tokenized_examples

In [45]:
tokenized_datasets = dataset.map(
    prepare_train_features, batched=True, remove_columns=dataset["train"].column_names
)

In [41]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
batch_size=64
model_dir = "./data/hf/models/distilbert-base-uncased-finetuned-squad"

args = TrainingArguments(
    output_dir=model_dir,
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [43]:
from transformers import default_data_collator

data_collator = default_data_collator

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipykernel_294/37394333.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=6177, training_loss=1.314632161952385, metrics={'train_runtime': 12376.3179, 'train_samples_per_second': 31.937, 'train_steps_per_second': 0.499, 'total_flos': 3.873165421863629e+16, 'train_loss': 1.314632161952385, 'epoch': 3.0})

In [23]:
model_to_save = trainer.save_model(model_dir)

In [53]:
import numpy as np
from collections import OrderedDict

raw_datasets = load_dataset("rajpurkar/squad_v2", cache_dir="./data/hf/datasets")
pad_on_right = tokenizer.padding_side == "right"

def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])
        tokenized_examples["offset_mapping"][i] = [
            o if sequence_id == (1 if pad_on_right else 0) else None
            for o, sequence_id in zip(
                tokenized_examples["offset_mapping"][i],
                tokenized_examples.sequence_ids(i)
            )
        ]
    return tokenized_examples

tokenized_validation = raw_datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [56]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForQuestionAnswering.from_pretrained(model_dir)

trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    eval_dataset=tokenized_datasets["validation"]
)

/tmp/ipykernel_19939/393521840.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [57]:
raw_predictions = trainer.predict(tokenized_validation)

In [58]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k["id"]: i for i, k in enumerate(examples)}
    features_per_example = [[] for _ in range(len(examples))]

    for i, feature in enumerate(features):
        example_index = example_id_to_index[feature["example_id"]]
        features_per_example[example_index].append(i)

    predictions = OrderedDict()

    for example_index, example in enumerate(examples):
        feature_indices = features_per_example[example_index]
        context = example["context"]
        valid_answers = []
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]
            input_ids = features[feature_index]["input_ids"]

            cls_index = input_ids.index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            start_indexes = np.argsort(start_logits)[-1: -n_best_size - 1: -1].tolist()
            end_indexes = np.argsort(end_logits)[-1: -n_best_size - 1: -1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append({
                        "score": start_logits[start_index] + end_logits[end_index],
                        "text": context[start_char: end_char]
                    })

        if valid_answers:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": ""}

        predictions[example["id"]] = best_answer["text"]

    return predictions

formatted_predictions = postprocess_qa_predictions(
    raw_datasets["validation"],
    tokenized_validation,
    raw_predictions.predictions
)

metric = evaluate.load("squad_v2")

references = [
    {"id": ex["id"], "answers": ex["answers"]}
    for ex in raw_datasets["validation"]
]

predictions_for_metric = [
    {"id": k, "prediction_text": v, "no_answer_probability": 0.0}
    for k, v in formatted_predictions.items()
]

results = metric.compute(predictions=predictions_for_metric, references=references)

print("Evaluation Results:")
print(f"F1: {results['f1']:.2f}")
print(f"Exact Match: {results['exact']:.2f}")

Evaluation Results:
F1: 41.57
Exact Match: 36.98
